# save and load  
Quelle: [tensorflow.org](https://www.tensorflow.org/tutorials/keras/save_and_load?hl=en)

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0-dev20210218


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 *28) / 255.0

In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer="adam", loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

In [4]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 7s 20ms/step - loss: 1.5699 - sparse_categorical_accuracy: 0.5318 - val_loss: 0.7176 - val_sparse_categorical_accuracy: 0.7790

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.4329 - sparse_categorical_accuracy: 0.8745 - val_loss: 0.5147 - val_sparse_categorical_accuracy: 0.8430

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2742 - sparse_categorical_accuracy: 0.9364 - val_loss: 0.4713 - val_sparse_categorical_accuracy: 0.8540

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1760 - sparse_categorical_accuracy: 0.9683 - val_loss: 0.4522 - val_sparse_categorical_accuracy: 0.8530

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1561 - sparse_cat

In [6]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3930 - sparse_categorical_accuracy: 0.1110
Untrained model, accuracy: 11.10%


In [10]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4071 - sparse_categorical_accuracy: 0.8720
Restored model, accuracy: 87.20%


In [11]:
checkpoint_path = "training_2/cp-{epoch:04d}.cpkt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5*batch_size)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback], validation_data=(test_images, test_labels), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.cpkt

Epoch 00010: saving model to training_2/cp-0010.cpkt

Epoch 00015: saving model to training_2/cp-0015.cpkt

Epoch 00020: saving model to training_2/cp-0020.cpkt

Epoch 00025: saving model to training_2/cp-0025.cpkt

Epoch 00030: saving model to training_2/cp-0030.cpkt

Epoch 00035: saving model to training_2/cp-0035.cpkt

Epoch 00040: saving model to training_2/cp-0040.cpkt

Epoch 00045: saving model to training_2/cp-0045.cpkt

Epoch 00050: saving model to training_2/cp-0050.cpkt


In [13]:
ls {checkpoint_dir}

checkpoint                        cp-0025.cpkt.index
cp-0000.cpkt.data-00000-of-00001  cp-0030.cpkt.data-00000-of-00001
cp-0000.cpkt.index                cp-0030.cpkt.index
cp-0005.cpkt.data-00000-of-00001  cp-0035.cpkt.data-00000-of-00001
cp-0005.cpkt.index                cp-0035.cpkt.index
cp-0010.cpkt.data-00000-of-00001  cp-0040.cpkt.data-00000-of-00001
cp-0010.cpkt.index                cp-0040.cpkt.index
cp-0015.cpkt.data-00000-of-00001  cp-0045.cpkt.data-00000-of-00001
cp-0015.cpkt.index                cp-0045.cpkt.index
cp-0020.cpkt.data-00000-of-00001  cp-0050.cpkt.data-00000-of-00001
cp-0020.cpkt.index                cp-0050.cpkt.index
cp-0025.cpkt.data-00000-of-00001


In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.cpkt'

In [15]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4920 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


In [21]:
model.save_weights("./checkpoints/my_checkpoint")

model = create_model()

model.load_weights("./checkpoints/my_checkpoint")

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4920 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


In [22]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save("saved_model/my_model")

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.6216 - sparse_categorical_accuracy: 0.4821
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4269 - sparse_categorical_accuracy: 0.8954
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2709 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2250 - sparse_categorical_accuracy: 0.9434
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1577 - sparse_categorical_accuracy: 0.9633
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [23]:
ls saved_model/

my_model/


In [24]:
ls saved_model/my_model/

assets/  keras_metadata.pb  saved_model.pb  variables/


In [25]:
new_model = tf.keras.models.load_model("saved_model/my_model")

new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}".format(100 * acc))
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4328 - sparse_categorical_accuracy: 0.8640
Restored model, accuracy: 86.40
(1000, 10)


In [27]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save("my_model.h5")

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.6503 - sparse_categorical_accuracy: 0.4907
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4308 - sparse_categorical_accuracy: 0.8776
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3073 - sparse_categorical_accuracy: 0.9244
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1895 - sparse_categorical_accuracy: 0.9551
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1519 - sparse_categorical_accuracy: 0.9764


In [28]:
new_model = tf.keras.models.load_model("my_model.h5")

new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [29]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4198 - sparse_categorical_accuracy: 0.8660
restored model, accuracy: 86.60%
